In [2]:
import pandas as pd
import plotly.express as px

In [3]:
time_df = pd.read_csv('data/programs_time_results_owl.mp4_3_nodemand.csv')

In [4]:
# take frame numbers
frame_numbers = time_df['frames_num'][0]

In [5]:
# average across the different iterations
mean_time_df = time_df.groupby(['p_version', 'n_workers'])['completion_time'].mean().reset_index()

In [6]:
# get the sequential time
sequential_time = mean_time_df[mean_time_df['p_version'] == 0]['completion_time'][0]
mean_time_df.drop(mean_time_df[mean_time_df['p_version'] == 0].index, inplace=True)

In [7]:
# add a speed up column
mean_time_df['speed_up'] = sequential_time / mean_time_df['completion_time']

In [8]:
# add ideal speed_up
for i in range(1,33):
    mean_time_df = mean_time_df.append({'p_version': "ideal", 'n_workers': i, 'speed_up':i}, ignore_index=True)

/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3168/11624701.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_time_df = mean_time_df.append({'p_version': "ideal", 'n_workers': i, 'speed_up':i}, ignore_index=True)
/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3168/11624701.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_time_df = mean_time_df.append({'p_version': "ideal", 'n_workers': i, 'speed_up':i}, ignore_index=True)
/var/folders/pg/z9tl77l52td7br14x19sr8n40000gn/T/ipykernel_3168/11624701.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mean_time_df = mean_time_df.append({'p_version': "ideal", 'n_workers': i, 'speed_up':i}, ignore_index=True)
/var/folders/pg/z9tl77l52td7br14x1

In [9]:
# top service time in milliseconds
program1_top_time = (mean_time_df[mean_time_df['p_version'] == 1].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
program2_top_time = (mean_time_df[mean_time_df['p_version'] == 2].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time
program3_top_time = (mean_time_df[mean_time_df['p_version'] == 3].sort_values(by='completion_time')['completion_time'].iloc[0])/(frame_numbers) # millisecondi di service time

program1_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 1].sort_values(by='completion_time')['n_workers'].iloc[0])
program2_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 2].sort_values(by='completion_time')['n_workers'].iloc[0])
program3_top_time_nworkers = (mean_time_df[mean_time_df['p_version'] == 3].sort_values(by='completion_time')['n_workers'].iloc[0])

In [10]:
mean_time_df['p_version'].replace({0: "sequential", 1: "thread", 2: "fast flow", 3: "fast flow 2" }, inplace=True)

In [11]:
fig = px.line(mean_time_df, x="n_workers", y="speed_up",color='p_version', title="Speedup", line_dash='p_version', ) 
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [12]:
# compute and show efficiency
mean_time_df['efficiency'] = mean_time_df['speed_up']/mean_time_df['n_workers']
fig = px.line(mean_time_df, x="n_workers", y="efficiency",color='p_version', title="Efficiency", line_dash='p_version')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [13]:
fig = px.line(mean_time_df.drop(mean_time_df[mean_time_df['p_version']=='ideal'].index), x="n_workers", y="completion_time",color='p_version', title="Completion Time", line_dash='p_version') 
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [14]:
print("Threaded top time = " + str(program1_top_time) + '. Using ' + str(program1_top_time_nworkers) + " workers")
print("FastFlow1 top time = " + str(program2_top_time) + '. Using ' + str(program2_top_time_nworkers) + " workers")
print("FastFlow2 top time = " + str(program3_top_time) + '. Using ' + str(program3_top_time_nworkers) + " workers")

Threaded top time = 9266.025099601593. Using 26 workers
FastFlow1 top time = 9168.680876494023. Using 26 workers
FastFlow2 top time = 38006.05498007969. Using 22 workers
